<a href="https://colab.research.google.com/github/simeonbetapudi/Harpeth/blob/main/ecoli_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install netCDF4
!pip install wget
!pip install pyproj

  Using cached netCDF4-1.6.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
  Using cached cftime-1.6.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=cd7fa6d84f6d2652def034290520c210a6ff186b567404499a19b9633a83a40d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import json
import pandas as pd
import requests
import numpy as np
from os import replace
import datetime
from datetime import datetime, timedelta
import math
import netCDF4 as nc
import wget
import pytz

In [ ]:
time = datetime.now(pytz.timezone('US/Central'))

month = time.month
year = time.year
day = time.day

mm = time.strftime('%m')
dd = time.strftime('%d')
stime = time.strftime('%Y%m%d')

weekago_month = (time - timedelta(days=7)).month
weekago_year = (time - timedelta(days=7)).year
weekago_day = (time - timedelta(days=7)).day

day_of_year = time.timetuple().tm_yday
B = 0.01721420632
C = 81.75
sindoy = math.sin(B * (day_of_year) - C)
sindoy_list = [sindoy]*8

Pulling precipitation data for each day from NWS.

In [ ]:
def get_dataset(url):
  retrieved_file = wget.download(url)
  dataset = nc.Dataset(retrieved_file)
  return dataset

def get_time(url):
  dataset = get_dataset(url)
  time_array = dataset['time'][:].toflex()
  time = time_array['_data'][()]
  datetime_obj = datetime.fromtimestamp(time)
  return datetime_obj.isoformat()

def create_prcp_array(dataset):
  prcp_array = dataset['observation'][473:487, 789:796]
  prcp_array = prcp_array.toflex()
  return prcp_array

def get_prcp(array):
  prcp = []
  xcrsfordf = []
  ycrsfordf = []
  for i in range (0, 14):
    for n in range (0, 7):
     prcp.append(array[i]['_data'][n])
     xcrsfordf.append(789 + n)
     ycrsfordf.append(473 + i)
  return prcp, xcrsfordf, ycrsfordf

def create_df(prcp, xcrsfordf, ycrsfordf):
  df = pd.DataFrame()
  df['Precipitation'] = prcp
  df['X-coordinates'] = xcrsfordf
  df['Y-coordinates'] = ycrsfordf
  return df

def find_prcp(dataframe, xcrs, ycrs):
  prcp_list = []
  for (a,b) in zip(xcrs, ycrs):
    value = dataframe.loc[dataframe['X-coordinates'].eq(a) & dataframe['Y-coordinates'].eq(b), 'Precipitation']
    prcp_value = value.values[0]
    prcp_list.append(prcp_value)
  return prcp_list

def pull_prcp(url, xcrs, ycrs):
  dataset = get_dataset(url)
  prcparray = create_prcp_array(dataset)
  prcp_info = get_prcp(prcparray)
  df = create_df(prcp_info[0], prcp_info[1], prcp_info[2])
  prcp_list = find_prcp(df, xcrs, ycrs)
  return prcp_list

In [ ]:
prcp_urls = ['https://water.weather.gov/precip/downloads/{}/{}/{}/nws_precip_1day_{}_conus.nc'.format(year, mm, dd, stime),
        'https://water.weather.gov/precip/downloads/{}/{}/{}/nws_precip_last2days_{}_conus.nc'.format(year, mm, dd, stime),
        'https://water.weather.gov/precip/downloads/{}/{}/{}/nws_precip_last3days_{}_conus.nc'.format(year, mm, dd, stime),
        'https://water.weather.gov/precip/downloads/{}/{}/{}/nws_precip_last5days_{}_conus.nc'.format(year, mm, dd, stime),
        'https://water.weather.gov/precip/downloads/{}/{}/{}/nws_precip_last7days_{}_conus.nc'.format(year, mm, dd, stime)]

xcrs = [792, 789, 792, 795, 795, 792, 792, 793]
ycrs = [484, 486, 479, 486, 476, 482, 478, 473]

In [ ]:
prcp_lists = []

for x in prcp_urls:
   prcp_list = pull_prcp(x, xcrs, ycrs)
   prcp_lists.append(prcp_list)

onedayprcp_list = prcp_lists[0]
twodayprcp_list = prcp_lists[1]
threedayprcp_list = prcp_lists[2]
fivedayprcp_list = prcp_lists[3]
weekprcp_list = prcp_lists[4]
#timestamp = get_time(urls[0])

In [ ]:
print(onedayprcp_list)
print(twodayprcp_list)
print(threedayprcp_list)
print(fivedayprcp_list)
print(weekprcp_list)
#print(timestamp)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Pulling temperature data from NWS's API.

In [ ]:
def get_temp_dict(url):
  request = (requests.get(url).text)
  temp_dict = json.loads(request)
  temp_dict = temp_dict['properties']['periods'][0]
  return temp_dict

def get_temp_df(temp_dict):
  temp_df = pd.DataFrame.from_dict(temp_dict)
  temp_df = temp_df[['temperature', 'probabilityOfPrecipitation','windSpeed', 'windDirection']]
  temp_df.columns = ['temperature', 'precipitation', 'windSpeed', 'windDirection']
  return temp_df

def get_temp_forecast(temp_df):
  temp_df['temperature'] = temp_df['temperature'].astype(str).astype(int)
  temp_forecast = temp_df['temperature'][1]
  return temp_forecast

def pull_temp(url):
  temp_dict = get_temp_dict(url)
  temp_df = get_temp_df(temp_dict)
  temp_forecast = get_temp_forecast(temp_df)
  return temp_forecast

In [ ]:
OHX_values = ['45,52', '39,54', '47,54', '48,45', '54,51', '48,55', '52,55', '46,50']

temp_urls = []

for x in OHX_values:
  temp_urls.append("https://api.weather.gov/gridpoints/OHX/{}/forecast".format(x))

In [ ]:
temp_list = []

for x in temp_urls:
  temp_forecast = pull_temp(x)
  temp_list.append(temp_forecast)

print(temp_list)

[97, 97, 98, 96, 96, 99, 98, 96]


Pulling Flow Data from USGS.

In [ ]:
flow_sites = ['03431700', '03433500', '03434500', '03432350', '03431060', '03434500', '03431700', '03431060']

flow_urls = []

for x in flow_sites:
  flow_urls.append("https://waterservices.usgs.gov/nwis/iv/?format=json&sites={}&startDT={}-{}-{}T04:00-0500&endDT={}-{}-{}T05:00-0500&parameterCd=00060&siteStatus=all".format(x, year, month, day, year, month, day).format(x))

In [ ]:
def pull_floworgage(url):
  data = json.loads(requests.get(url).text)
  data = data['value']['timeSeries'][0]['values'][0]['value']
  df = pd.DataFrame.from_dict(data)
  df['value'] = df['value'].astype(str).astype(float)
  floworgage = df['value'].mean()
  return floworgage

In [ ]:
flow_list = []

for x in flow_urls:
  flow = pull_floworgage(x)
  flow_list.append(flow)

print(flow_list)

[8.02, 157.0, 244.0, 38.6, 25.32, 244.0, 8.02, 25.32]


Gage Height

In [ ]:
gage_sites = ['03431700', '03433500', '03434500', '03432350', '03431060', '03434500', '03431700', '03431060']

gage_urls = []

for x in gage_sites:
  gage_urls.append("https://waterservices.usgs.gov/nwis/iv/?format=json&sites={}&startDT={}-{}-{}T04:00-0500&endDT={}-{}-{}T05:00-0500&parameterCd=00065&siteStatus=all".format(x, year, month, day, year, month, day).format(x))

In [ ]:
gage_list = []

for x in gage_urls:
  gage = pull_floworgage(x)
  gage_list.append(gage)

print(gage_list)

[0.7, 1.67, 1.633333333333333, 3.79, 2.564, 1.633333333333333, 0.7, 2.564]


Creating a DataFrame of Forecast Values

In [ ]:
column_names = ['temp', 'onedayprcp', 'twodayprcp', 'threedayprcp', 'fivedayprcp', 'weekprcp', 'flow', 'gage', 'sin(dayofyear)']

In [ ]:
index_names = ['Hwy100', 'Hwy70', 'JBlvd', 'LPike', 'MCreek', 'MBridge', 'RCreek', 'WPark']

In [ ]:
forecast_df = pd.DataFrame(list(zip(temp_list, onedayprcp_list, twodayprcp_list, threedayprcp_list, fivedayprcp_list, weekprcp_list, flow_list, gage_list, sindoy_list)), index = index_names, columns = column_names)

In [ ]:
forecast_df

,temp,onedayprcp,twodayprcp,threedayprcp,fivedayprcp,weekprcp,flow,gage,sin(dayofyear)
Hwy100,97,0.0,0.0,0.0,0.0,0.0,8.02,0.700000,-0.7414
Hwy70,97,0.0,0.0,0.0,0.0,0.0,157.00,1.670000,-0.7414
JBlvd,98,0.0,0.0,0.0,0.0,0.0,244.00,1.633333,-0.7414
LPike,96,0.0,0.0,0.0,0.0,0.0,38.60,3.790000,-0.7414
MCreek,96,0.0,0.0,0.0,0.0,0.0,25.32,2.564000,-0.7414
MBridge,99,0.0,0.0,0.0,0.0,0.0,244.00,1.633333,-0.7414
RCreek,98,0.0,0.0,0.0,0.0,0.0,8.02,0.700000,-0.7414
WPark,96,0.0,0.0,0.0,0.0,0.0,25.32,2.564000,-0.7414


In [ ]:
timestamp = get_time(prcp_urls[0])
print(timestamp)

2023-08-23T07:00:02
